## Creating and solving Application 1 with Gurobi

### Restaurant Capacity Maximization Problem



Note: Refer to readme file for run instructions

**Decision Variables**

$f_i$ : 1 if table i  is used, 0 otherwise, where $i=1...N$

$g_{ij}:$ 1 if both table i and j are used, 0 otherwise,  

$x_i =$ bottom left x coordinate of table i

$y_i =$ bottom left x coordinate of table i

$l_{ij} =$ 1 if table i is located left to j

$b_{ij} =$ 1 if table i is located below j


**Parameters**

$d$ : The minimum physical distance required between tables

$c_i =$ max # people that can be seated @ table i, where $i=1...N$

$C$ = Minimum desired establishment capacity

$w_i =$ width of table i + d

$h_i =$ height of table i + d

$R_w =$ width of dining space

$R_h =$ height of dining space

$U_{ij}$ = Underperformance measure of staff when table i and j are used


**Objective Function**

$min$ $z$ = $g_1^+ + g_2^- +\sum_{i=1}^{N}\sum_{j=1, i\neq j}^{N}g_{ij}U_{ij}$


**Constraints**

**s.t**
Force non-linear element to be 0 or 1:

$g_{ij} >= f_i+f_j-1 \ i, j \in T and \ i \neq j$

No overlap if the following constraint is true:

$l_{ij} + l_{ji} + b_{ij} + b_{ji} + (1-x_i) + (1-x_j) >= 1, \ i, j \in T, \ i<j$ 

If a table i is to the left of table j, table i's right edge x coordinate  must be less than table j's x left edge x coordinate:

$x_i +w_i <= x_j + W - Wl_{ij}, \ i, j \in T$

If table i is below j, table i's top edge y coordinate must be less than table j's bottom edge y coordinate:

$y_i +h_i <= y_j + H - Hb_{ij}, \ i, j \in T$

Tables must exist within the dining space:

$x_i >= 0, \ i \in T$

$x_i+w_i <= R_x, \ i \in T$

$y_i >= 0, \ i \in T$

$y_i+h_i <= R_h, \ i \in T$

Unused table space must be under U:

$R_wR_h \ - \sum_{i=1}^{N}f_i(w_i)(h_i) - (g_1^+-g_1^-) = U, \ i \in T$ 

Facility capacity must be over C to remain profitable

$\sum_{i=1}^{N}c_if_i - (g_2^+-g_2^-) = C, \ i \in T$

Binary variable constraints:

$l_{ij}, b_{ij} \ \in {0,1}$

$f_{i} \ \in {0,1}$

$g_{ij}\in {0,1}$

## Solution of sample instance
First, we will use a small instance to demonstrate the individual steps that the programmatic approach uses to solve the generated instances

#### Step 1: Import grobipy module

In [112]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from gurobipy import quicksum
import plotly.graph_objects as go

#### Step 2: Define the model

In [113]:
m = gp.Model()

#### Step 3: Import generated instance data

In [114]:
# this step is done in the programmatic looping solution

#### Step 4: Define the parameters

In [115]:
# number of tables
N = 3

# Minimum physical distane required between tables
d = 2

# capacity for each table
c = [2, 2, 2]

# width for each table
w = [1, 1, 2]

# account for physical distancing margin
for i in range(len(w)):
    w[i] += d

# height for each table
h = [1, 2, 2]

# account for physical distancing margin
for i in range(len(h)):
    h[i] += d

# Width of dining space
Rw = 7

# Height of dining space
Rh = 7

# Unused table space goal
U = 2

# Minimum desired establishment capacity goal
C = 10

# Underperformance measure of staff when table i and j are used Uij[rows][columns]
Uij =  [[1,1,1],
        [1,1,1],
        [1,1,1]]

#### Step 5: Define decision variables

In [116]:
# binary, if table i is used
f = {}
for i in range(N):
    f[i] = m.addVar(vtype=GRB.BINARY, name="f_" +str(i))

# bottom left x coordinate of table i
x = {}
for i in range(N):
    x[i] = m.addVar(vtype=GRB.CONTINUOUS, name="x_"+str(i))
    
# bottom left y coordinate of table i
y = {}
for i in range(N):
    y[i] = m.addVar(vtype=GRB.CONTINUOUS, name="y_"+str(i))
    
    
# binary, if table i is located left to table j
l = {}
for i in range(N):
    for j in range(N):
        if i != j:
            l[i, j] = m.addVar(vtype=GRB.BINARY, name="l_" +str(i)+str(j))

# binary, if table i is located below to table j
b = {}
for i in range(N):
    for j in range(N):
        if i != j:
            b[i, j] = m.addVar(vtype=GRB.BINARY, name="b_" +str(i)+str(j))

# binary, if table i and table j are both used
g = {}
for i in range(N):
    for j in range(N):
        if i != j:
            g[i, j] = m.addVar(vtype=GRB.BINARY, name="g_" +str(i)+str(j))

# surplus for goal 1  
g1surplus={}
g1surplus = m.addVar(vtype=GRB.CONTINUOUS, name="g1surplus")
    
# slack for goal 1  
g1slack={}
g1slack = m.addVar(vtype=GRB.CONTINUOUS, name="g1slack")

# surplus for goal 2
g2surplus={}
g2surplus= m.addVar(vtype=GRB.CONTINUOUS, name="g2surplus")

# slack/surplus for goal 2
g2slack={}
g2slack= m.addVar(vtype=GRB.CONTINUOUS, name="g2slack")

#### Step 6: Set the objective function

In [117]:
nonlinearobj = quicksum((g[i, j]*Uij[i][j])/2 for i in range(N) for j in range(N) if i != j)
m.setObjective(g1surplus+g2slack+(nonlinearobj), GRB.MINIMIZE)

#### Step 7: Add constraints

In [118]:
# no overlap
m.addConstrs(l[i,j] + l[j,i] + b[i,j] + b[j,i] + (1-f[i]) + (1-f[j])  >= 1 for i in range(N) for j in range(N) if i != j)

# If a table i is to the left of table j, table i's right edge x coordinate  must be less than table j's x left edge x coordinate
m.addConstrs(x[i] + w[i] <= (x[j] + Rw - Rw*l[i,j]) for i in range(N) for j in range(N) if i != j)

# If table i is below j, table i's top edge y coordinate must be less than table j's bottom edge y coordinate
m.addConstrs(y[i] + h[i] <= (y[j] + Rh - Rh*b[i,j]) for i in range(N) for j in range(N) if i != j)

# table space must exist within the dining space
m.addConstrs(x[i] >= 0 for i in range(N))
m.addConstrs(x[i]+w[i]<= Rw for i in range (N))
m.addConstrs(y[i] >= 0 for i in range(N))
m.addConstrs(y[i]+h[i]<= Rh for i in range (N))

# Unused table space must be under U - GOAL 1
m.addConstrs((Rw*Rh) - quicksum((w[i])*(h[i])*f[i] for i in range(N)) - g1surplus + g1slack == U for i in range(N))

# Facility capacity must be over C to remain profitable - GOAL 2
m.addConstrs(quicksum(f[i]*c[i] for i in range(N)) + g2slack - g2surplus == C for i in range(N))

# Quadratic constraint
m.addConstrs(g[i,j] >= f[i] + f[j] - 1 for i in range(N) for j in range(N) if i != j)

# update model
m.update()

#### Step 8: Solve the model

In [119]:
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 42 rows, 31 columns and 132 nonzeros
Model fingerprint: 0xb5d3c80a
Variable types: 10 continuous, 21 integer (21 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 39.0000000
Presolve removed 26 rows and 11 columns
Presolve time: 0.00s
Presolved: 16 rows, 20 columns, 53 nonzeros
Variable types: 3 continuous, 17 integer (17 binary)

Root relaxation: objective 1.700000e+01, 6 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   17.00000    0    6   39.00000   17.00000  56.4%     -    0s
H    0     0                      29.0000000   17.00000  41.4%     -    0s
H    0     0                      17.0000000   17.00000  0.00%    

#### Insert variable values

In [120]:
# Parameters
print("Room size:", Rw*Rh, "m^2")
# Tables:
for i in f:
    print("Table", i, "was", "not" if i==0 else "", "used.", "Position:", "(", x[i].x,",",y[i].x,").", "Capacity:", c[i], "Table space height:", h[i], "Table space width:", w[i])
# Goal 1 print
print("Goal 1: Unused space must be under", U, "m^2.", )
print("Goal 1 Result: Unused space was", g1surplus.x, "m^2.", )
# Goal 2 print
print("Goal 2: Facility capacity must be over", C, "to remain profitable.", )
print("Goal 2 Result: Facility capacity was unmet by", g2slack.x, ".", )
# nonlinear element print
print("Underperformance level for this table setup is", nonlinearobj.getValue())
tableUsedData = []
for tableUsed in f:
    tableUsedData.append(f[tableUsed].x)
xCoords = []
for i in x:
    xCoords.append(x[i].x)
yCoords = []
for i in y:
    yCoords.append(y[i].x)
d = {'ifUsed' : tableUsedData, 'x-coord' : xCoords, 'y-coord' : yCoords, 'capacity' : c, 'table space width' : w, 'table space height' : h}
df = pd.DataFrame(data=d)
df



Room size: 49 m^2
Table 0 was not used. Position: ( 0.0 , 0.0 ). Capacity: 2 Table space height: 3 Table space width: 3
Table 1 was  used. Position: ( 4.0 , 0.0 ). Capacity: 2 Table space height: 4 Table space width: 3
Table 2 was  used. Position: ( 0.0 , 3.0 ). Capacity: 2 Table space height: 4 Table space width: 4
Goal 1: Unused space must be under 2 m^2.
Goal 1 Result: Unused space was 10.0 m^2.
Goal 2: Facility capacity must be over 10 to remain profitable.
Goal 2 Result: Facility capacity was unmet by 4.0 .
Underperformance level for this table setup is 3.0


,ifUsed,x-coord,y-coord,capacity,table space width,table space height
0,1.0,0.0,0.0,2,3,3
1,1.0,4.0,0.0,2,3,4
2,1.0,0.0,3.0,2,4,4


In [121]:
xCoordPlot = []
yCoordPlot = []
for i in f:
    if(f[i].x == 1):
        #top border
        xCoordPlot.append(0)
        xCoordPlot.append(Rw)
        yCoordPlot.append(Rh)
        yCoordPlot.append(Rh)
        xCoordPlot.append("none")
        yCoordPlot.append("none")
        
        #right border
        xCoordPlot.append(Rw)
        xCoordPlot.append(Rw)
        yCoordPlot.append(0)
        yCoordPlot.append(Rh)
        xCoordPlot.append("none")
        yCoordPlot.append("none")
        #x
        xCoordPlot.append(x[i].x)
        xCoordPlot.append(x[i].x)
        xCoordPlot.append(x[i].x + w[i])
        xCoordPlot.append(x[i].x + w[i])
        xCoordPlot.append(x[i].x)
        xCoordPlot.append("none")
        #y
        yCoordPlot.append(y[i].x)
        yCoordPlot.append(y[i].x + h[i])
        yCoordPlot.append(y[i].x + h[i])
        yCoordPlot.append(y[i].x)
        yCoordPlot.append(y[i].x)
        yCoordPlot.append("none")
fig = go.Figure(go.Scatter(x=xCoordPlot, y=yCoordPlot, fill="toself"))
fig.show()

## Programmatic looping solution

Here, we will be importing our previously generated instance data. This data exists in JSON format, and is easily loaded in a useable Dict format by using the python json module. We will be solving all of these instances, and the solution data including the generated model, solution data, and decision variable values will be outputted to the '/solved_models/' directory.

### Read in generated instances

In [122]:
import json 
with open('generated_instances.json') as f:
  loaded_instances = json.load(f)

### Define a function to wrap the entire solving flow from above section

In [125]:
# instance is an object containing generated instance parameters
def solveModel(instance, iteration):
    print("INSTANCE #", iteration)
    # STEP 1: DEFINE THE MODEL
    m = gp.Model()
    
    # STEP 2: DEFINE THE PARAMETERS THAT WILL BE USED
    
    # number of tables
    N = instance['num_tables']
    print("Number of tables:", N)
    
    # Minimum physical distance required between tables
    d = 2
    
    # capacity for each table
    c = [table['capacity'] for table in instance['tables']]
    
    # width for each table
    w = [table['width'] for table in instance['tables']]
    print("Table widths:", w)
   
    # account for physical distancing margin
    for i in range(len(w)):
        w[i] += d

    # height for each table
    h = [table['height'] for table in instance['tables']]
    print("Table heights:", h)
    
    # account for physical distancing margin
    for i in range(len(h)):
        h[i] += d
    
    # Width of dining space
    Rw = instance['width']

    # Height of dining space
    Rh = instance['height']
    
    # Unused table space goal
    U = 20

    # Minimum desired establishment capacity goal
    # For the purposes of this assignment, C is assumed to be half of the maximum capacity of an instance. 
    # In a real scenario, the owner of the restaurant would provide C as a parameter
    C = (sum(c) // 2)
    print ('Minimum desired capacity: %d' % C)
    
    # Underperformance measure of staff when table i and j are used Uij[rows][columns]
    # For the purposes of this assignment, the Uij matrix is assumed to have an equal value for all indices. 
    # In a real scenario, the restaurant owner would provide this matrix as a parameter
    Uij =  [[1 for table in instance['tables']] for table in instance['tables'] ]
    print("Table underperformance matrix:", Uij)
  
    # STEP 3: DEFINE DECISION VARIABLES
    
    # binary, if table i is used
    f = {}
    for i in range(N):
        f[i] = m.addVar(vtype=GRB.BINARY, name="f_" +str(i))

    # bottom left x coordinate of table i
    x = {}
    for i in range(N):
        x[i] = m.addVar(vtype=GRB.CONTINUOUS, name="x_"+str(i))

    # bottom left y coordinate of table i
    y = {}
    for i in range(N):
        y[i] = m.addVar(vtype=GRB.CONTINUOUS, name="y_"+str(i))


    #binary, if table i is located left to table j
    l = {}
    for i in range(N):
        for j in range(N):
            if i != j:
                l[i, j] = m.addVar(vtype=GRB.BINARY, name="l_" +str(i)+str(j))

    # binary, if table i is located below to table j
    b = {}
    for i in range(N):
        for j in range(N):
            if i != j:
                b[i, j] = m.addVar(vtype=GRB.BINARY, name="b_" +str(i)+str(j))

    # binary, if table i and table j are both used
    g = {}
    for i in range(N):
        for j in range(N):
            if i != j:
                g[i, j] = m.addVar(vtype=GRB.BINARY, name="g_" +str(i)+str(j))

    # surplus for goal 1  
    g1surplus={}
    g1surplus = m.addVar(vtype=GRB.CONTINUOUS, name="g1surplus")
    
    # slack for goal 1  
    g1slack={}
    g1slack = m.addVar(vtype=GRB.CONTINUOUS, name="g1slack")
    
    
    # surplus for goal 2
    g2surplus={}
    g2surplus= m.addVar(vtype=GRB.CONTINUOUS, name="g2surplus")

    # slack/surplus for goal 2
    g2slack={}
    g2slack= m.addVar(vtype=GRB.CONTINUOUS, name="g2slack")
    
    
    # STEP 4: DEFINE OBJECTIVE FUNCTION
    nonlinearobj = quicksum((g[i, j]*Uij[i][j])/2 for i in range(N) for j in range(N) if i != j)
    m.setObjective(g1surplus+g2slack+(nonlinearobj), GRB.MINIMIZE)
    
    # Step 5: ADD CONSTRAINTS
    # no overlap
    m.addConstrs(l[i,j] + l[j,i] + b[i,j] + b[j,i] + (1-f[i]) + (1-f[j])  >= 1 for i in range(N) for j in range(N) if i != j)

    # If a table i is to the left of table j, table i's right edge x coordinate  must be less than table j's x left edge x coordinate
    m.addConstrs(x[i] + w[i] <= (x[j] + Rw - Rw*l[i,j]) for i in range(N) for j in range(N) if i != j)

    # If table i is below j, table i's top edge y coordinate must be less than table j's bottom edge y coordinate
    m.addConstrs(y[i] + h[i] <= (y[j] + Rh - Rh*b[i,j]) for i in range(N) for j in range(N) if i != j)

    # table space must exist within the dining space
    m.addConstrs(x[i] >= 0 for i in range(N))
    m.addConstrs(x[i]+w[i]<= Rw for i in range (N))
    m.addConstrs(y[i] >= 0 for i in range(N))
    m.addConstrs(y[i]+h[i]<= Rh for i in range (N))

    # Unused table space must be under U - GOAL 1
    m.addConstrs((Rw*Rh) - quicksum((w[i])*(h[i])*f[i] for i in range(N)) - g1surplus + g1slack == U for i in range(N))

    # Facility capacity must be over C to remain profitable - GOAL 2
    m.addConstrs(quicksum(f[i]*c[i] for i in range(N)) + g2slack -g2surplus == C for i in range(N))

    # Quadratic constraint
    m.addConstrs(g[i,j] >= f[i] + f[j] - 1 for i in range(N) for j in range(N) if i != j)

    # update model
    m.update()
    
    # STEP 6: SOLVE THE MODEL
    m.optimize()
    
    # STEP 7: VISUALLY DISPLAY RESULTS
    # Parameters
    print("Room size:", Rw*Rh, "m^2")
    # Tables:
    for i in f:
        print("Table", i, "was", "not" if i==0 else "", "used.", "Position:", "(", x[i].x,",",y[i].x,").", "Capacity:", c[i], "Table space height:", h[i], "Table space width:", w[i])
    # Goal 1 print
    print("Goal 1: Unused space must be under", U, "m^2.", )
    print("Goal 1 Result: Unused space was", g1surplus.x, "m^2.", )
    
    # Goal 2 print
    print("Goal 2: Facility capacity must be over", C, "to remain profitable.", )
    print("Goal 2 Result: Facility capacity was unmet by", g2slack.x, ".", )
    
    # nonlinear element print
    print("Underperformance level for this table setup is", nonlinearobj.getValue())
    tableUsedData = []
    
    for tableUsed in f:
        tableUsedData.append(f[tableUsed].x)
    xCoords = []
    for i in x:
        xCoords.append(x[i].x)
    yCoords = []
    for i in y:
        yCoords.append(y[i].x)
    d = {'ifUsed' : tableUsedData, 'x-coord' : xCoords, 'y-coord' : yCoords, 'capacity' : c, 'table space width' : w, 'table space height' : h}
    df = pd.DataFrame(data=d)
    df
    
    

In [123]:
# STEP 8: CREATE VISUALIZATION USING PLOTLY
    xCoordPlot = []
    yCoordPlot = []
    for i in f:
        if(f[i].x == 1):
            #top border
            xCoordPlot.append(0)
            xCoordPlot.append(Rw)
            yCoordPlot.append(Rh)
            yCoordPlot.append(Rh)
            xCoordPlot.append("none")
            yCoordPlot.append("none")

            #right border
            xCoordPlot.append(Rw)
            xCoordPlot.append(Rw)
            yCoordPlot.append(0)
            yCoordPlot.append(Rh)
            xCoordPlot.append("none")
            yCoordPlot.append("none")
            #x
            xCoordPlot.append(x[i].x)
            xCoordPlot.append(x[i].x)
            xCoordPlot.append(x[i].x + w[i])
            xCoordPlot.append(x[i].x + w[i])
            xCoordPlot.append(x[i].x)
            xCoordPlot.append("none")
            #y
            yCoordPlot.append(y[i].x)
            yCoordPlot.append(y[i].x + h[i])
            yCoordPlot.append(y[i].x + h[i])
            yCoordPlot.append(y[i].x)
            yCoordPlot.append(y[i].x)
            yCoordPlot.append("none")
            
    fig = go.Figure(go.Scatter(x=xCoordPlot, y=yCoordPlot, fill="toself"))
    #fig.show()
    
    # STEP 9: SAVE MODEL, SOLUTION, PLOTLY FIGURE, AND DECISION VARIABLE VALUES TO /solved_models/ DIRECTORY
    #m.write('solved_models/instance_%d_out.lp' % iteration)
    #m.write('solved_models/instance_%d_out.json' % iteration)
    #fig.write_image('solved_models/instance_%d_fig.png' % iteration, engine='kaleido')
    #df.to_csv('solved_models/instance_%d_out.csv' % iteration, index=False)
    

### Solve all instances

In [124]:
iteration = 0
for instance in loaded_instances:
    solveModel(instance, iteration)
    iteration+=1

INSTANCE # 0
Number of tables: 5
Table widths: [4, 3, 3, 2, 2]
Table heights: [3, 3, 4, 2, 2]
Minimum desired capacity: 28
Table underperformance matrix: [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 110 rows, 79 columns and 390 nonzeros
Model fingerprint: 0x6b867500
Variable types: 14 continuous, 65 integer (65 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 203.0000000
Presolve removed 49 rows and 13 columns
Presolve time: 0.00s
Presolved: 61 rows, 66 columns, 216 nonzeros
Variable types: 10 continuous, 56 integer (55 binary)

Root relaxation: objective 6.800000e+01, 19 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    

Optimize a model with 110 rows, 79 columns and 390 nonzeros
Model fingerprint: 0xc81cb5e6
Variable types: 14 continuous, 65 integer (65 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 158.0000000
Presolve removed 48 rows and 12 columns
Presolve time: 0.00s
Presolved: 62 rows, 67 columns, 222 nonzeros
Variable types: 10 continuous, 57 integer (56 binary)

Root relaxation: objective 1.000000e+01, 20 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   10.00000    0    2  158.00000   10.00000  93.7%     -    0s
H    0     0                      13.0000000   10.00000  23.1%     -    0s
     0     0   10.00000    0    2   13.00000   10.00000  23.1%     -    0s
     0     0   10.00000    0    2   13.00


Optimal solution found (tolerance 1.00e-04)
Best objective 2.054000000000e+03, best bound 2.054000000000e+03, gap 0.0000%
Room size: 2352 m^2
Table 0 was not used. Position: ( 0.0 , 13.0 ). Capacity: 16 Table space height: 6 Table space width: 6
Table 1 was  used. Position: ( 0.0 , 19.0 ). Capacity: 8 Table space height: 4 Table space width: 4
Table 2 was  used. Position: ( 0.0 , 0.0 ). Capacity: 12 Table space height: 5 Table space width: 5
Table 3 was  used. Position: ( 5.0 , 27.0 ). Capacity: 12 Table space height: 5 Table space width: 5
Table 4 was  used. Position: ( 0.0 , 23.0 ). Capacity: 12 Table space height: 5 Table space width: 5
Table 5 was  used. Position: ( 6.0 , 22.0 ). Capacity: 12 Table space height: 5 Table space width: 5
Table 6 was  used. Position: ( 6.0 , 13.0 ). Capacity: 12 Table space height: 6 Table space width: 5
Table 7 was  used. Position: ( 11.0 , 13.0 ). Capacity: 12 Table space height: 5 Table space width: 5
Table 8 was  used. Position: ( 6.0 , 8.0 ). Cap

Model fingerprint: 0x45bef8b1
Variable types: 32 continuous, 560 integer (560 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 1429.0000000
Presolve removed 265 rows and 94 columns
Presolve time: 0.01s
Presolved: 547 rows, 498 columns, 1926 nonzeros
Variable types: 28 continuous, 470 integer (469 binary)

Root relaxation: objective 1.150000e+03, 162 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                    1150.0000000 1059.00000  7.91%     -    0s
     0     0          -    0      1150.00000 1150.00000  0.00%     -    0s

Explored 0 nodes (360 simplex iterations) in 0.04 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 1150 1429 

Optimal solution found (toler

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 1056 rows, 772 columns and 4240 nonzeros
Model fingerprint: 0x36b2bf7f
Variable types: 36 continuous, 736 integer (736 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 1863.0000000
Presolve removed 335 rows and 123 columns
Presolve time: 0.01s
Presolved: 721 rows, 649 columns, 2537 nonzeros
Variable types: 32 continuous, 617 integer (616 binary)

Root relaxation: objective 1.559000e+03, 261 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                    1559.0000000 1439.00000  7.70%     -    0s
     0     0          -    0      1559.00000 1559.00000  0.00%     -    0s

Explored 0 nodes (453 simplex iterations) in 0

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 1332 rows, 976 columns and 5382 nonzeros
Model fingerprint: 0x67d812b6
Variable types: 40 continuous, 936 integer (936 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 2184.0000000
Presolve removed 413 rows and 156 columns
Presolve time: 0.01s
Presolved: 919 rows, 820 columns, 3232 nonzeros
Variable types: 36 continuous, 784 integer (783 binary)

Root relaxation: objective 1.743000e+03, 227 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1743.00000    0   61 2184.00000 1743.00000  20.2%     -    0s
H    0     0                    1753.0000000 1743.00000  0.57%     -    0s
H    0     0                    1744.0000000 17

## Solutions to models also available in '/solved_models/' directory